In [ ]:
async def run_test():
    """Build the agent and run it with the prompt."""
    print(f"Building agent...")
    
    # Build the agent graph
    checkpointer = await get_checkpointer()
    graph = await build_agent(checkpointer=checkpointer)
    print("✓ Agent built\n")
    
    # Create messages
    messages = [HumanMessage(content=PROMPT)]
    
    print("Running agent...")
    print("-" * 60)
    
    # Run the agent and capture events
    event_count = 0
    async for event_type, data in run_agent(graph, messages, cwd=WORKSPACE_PATH):
        event_count += 1
        
        if event_type == "intent":
            print(f"\n[INTENT] {data.type}: {data.category}")
        
        elif event_type == "task_list":
            print(f"\n[TASK LIST]")
            for i, task in enumerate(data, 1):
                print(f"  {i}. {task}")
        
        elif event_type == "task_start":
            print(f"\n[TASK START] {data}")
        
        elif event_type == "task_complete":
            print(f"[TASK COMPLETE] {data}")
        
        elif event_type == "assistant":
            print(f"\n[ASSISTANT]\n{data}")
        
        elif event_type == "tool_call":
            print(f"\n[TOOL CALL] {data}")
        
        elif event_type == "tool_result":
            # Truncate long tool results for readability
            result = str(data)
            if len(result) > 500:
                result = result[:500] + "\n... (truncated)"
            print(f"[TOOL RESULT]\n{result}")
        
        elif event_type == "error":
            print(f"\n[ERROR] {data}")
        
        elif event_type == "messages":
            # Skip printing full message history
            pass
    
    print("\n" + "-" * 60)
    print(f"✓ Agent completed ({event_count} events)")

# Run the test
await run_test()

In [ ]:
# Configuration
WORKSPACE_PATH = os.path.expanduser("~/dev")  # Change this to your workspace
PROMPT = "List all Rust files in the project"  # Change this to your prompt

print(f"Workspace: {WORKSPACE_PATH}")
print(f"Prompt: {PROMPT}")

In [ ]:
import asyncio
import os
import sys
from pathlib import Path

# Add axono to path
sys.path.insert(0, str(Path.cwd().parent))

from axono.agent import build_agent, run_agent
from axono.conversation import get_checkpointer
from langchain_core.messages import HumanMessage

# Axono Agent Testing Notebook

Test the axono langgraph agent separately by specifying a workspace path and running a prompt.

This notebook allows you to:
- Set a workspace path
- Write a prompt
- Run the agent step-by-step
- See the graph execution and tool calls